<a href="https://colab.research.google.com/github/tirsoad/03MIAR---Algotritmos-de-Optimizacion---2023/blob/master/TrabajoPr%C3%A1cticoAlgoritmos_TirsoAiraDominguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Tirso Aira Domínguez  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en lastomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todosla misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente delnúmero de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planifi car lassesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?



1.   **Como represento el espacio de soluciones:**
    El espacio de soluciones se representa utilizando permutaciones de los números del 0 al 9. Cada permutación representa una asignación de actores a tomas. Por ejemplo, la solución [2, 9, 0, 6, 4, 8, 5, 3, 7, 1] significa que el actor 0 está asignado a la toma 2, el actor 1 a la toma 9, y así sucesivamente.
2.   **Cual es la función objetivo:**
    La función objetivo se define en la función f_coste(sol, matriz). Esta función calcula el costo total de una solución dada, sumando los valores correspondientes de la matriz de participación para cada asignación de actor a toma.
3.    **Como implemento las restricciones:**
    La función crear_hijos(nodo, n) genera hijos para un nodo dado excluyendo aquellos actores que ya están asignados en el nodo actual. Esto garantiza que no se violen restricciones de asignación duplicada de actores a tomas.
    Además, el algoritmo de ramificación y poda se encarga de explorar solo las soluciones prometedoras, evitando explorar ramas que claramente no conducen a una solución óptima.





In [2]:
#Respuesta

## IMPORTAMOS LIBRERIAS
import pandas as pd
from google.colab import files
import numpy as np
import random


In [3]:
## CARGA Y VISUALIZACION DEL DATASET

# Se importa el archivo .csv
csv_file = files.upload()

# Paso necesario para poder leer el csv de forma correcta
dataframe = list(csv_file.keys())[0]

# Cargar, procesar y visualizar el dataframe
df = pd.read_csv(dataframe, index_col=0)
# Se elimina la columna 11 ya que es una columna vacia que no aporta información
df = df.drop('Unnamed: 11', axis=1)
print(df)


Saving DataSet_ActoresDoblaje.csv to DataSet_ActoresDoblaje.csv
       Actor  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  \
Toma     1.0         2.0         3.0         4.0         5.0         6.0   
1        1.0         1.0         1.0         1.0         1.0         0.0   
2        0.0         0.0         1.0         1.0         1.0         0.0   
3        0.0         1.0         0.0         0.0         1.0         0.0   
4        1.0         1.0         0.0         0.0         0.0         0.0   
5        0.0         1.0         0.0         1.0         0.0         0.0   
6        1.0         1.0         0.0         1.0         1.0         0.0   
7        1.0         1.0         0.0         1.0         1.0         0.0   
8        1.0         1.0         0.0         0.0         0.0         1.0   
9        1.0         1.0         0.0         1.0         0.0         0.0   
10       1.0         1.0         0.0         0.0         0.0         1.0   
11       1.0         1.0

In [4]:
## DESARROLLO DEL PROBLEMA I - CREACION DE FUNCIONES AUXILIARES

matriz_participacion = df.iloc[1:30, 0:10].astype(int).values
solucion_parcial_elegida = np.array([2, 9, 0, 6, 4, 8, 5, 3, 7, 1])

def f_coste(sol,matriz):
  val = 0
  for i in range (len(sol)):
    val += matriz[sol[i]][i]
  return val

def f_cota_inferior(sol,matriz):
  val = 0
  for i in range (len(sol)):
    val += matriz[i][sol[i]]

  for i in range (len(sol), matriz.shape[1]):
    val += np.min(matriz[:, 1])

  return val

def crear_hijos(nodo, n):
  hijos = []
  for i in range(n):
    if i not in nodo:
      hijos.append({'s': nodo + (i,)})
  return hijos

In [ ]:
## DESARROLLO DEL PROBLEMA I - FUNCION MAIN

# dim = len(matriz_participacion[0])
# sol_optima = tuple(i for i in range(dim))
# cota_superior = f_coste(sol_optima, matriz_participacion)
# nodos = []
# nodos.append({'s': (), 'ci': f_cota_inferior((), matriz_participacion)})
# iteracion = 0
# while len(nodos) > 0:
#   iteracion += 1
#   nodo_prometedor = min(nodos, key=lambda x: x['ci'])['s']

#   # Se generan los hijos:
#   hijos = [{'s': x['s'], 'ci': f_cota_inferior(x['s'], matriz_participacion)} for x in crear_hijos(nodo_prometedor, dim)]

#   # Revisamos la cota superior y nos quedamos con la mejor solución si llegamos a una solución final:
#   nodo_final = [x for x in hijos if len(x['s']) == dim]
#   if len(nodo_final) > 0:
#     if nodo_final[0]['ci'] < cota_superior:
#       cota_superior = nodo_final[0]['ci']
#       sol_optima = nodo_final[0]['s']
#   # Poda:
#   hijos = [x for x in hijos if x['ci'] < cota_superior]
#   # Añadimos los hijos:
#   nodos.extend(hijos)
#   # Eliminamos el nodo ramificado:
#   nodos = [x for x in nodos if x['s'] != nodo_prometedor]

# print("La solución final es:", sol_optima, "en", iteracion, "iteraciones", "para dimension:", dim)

# # # Usamos nuestra matriz de costos para actores.
# # ramificacion_y_poda(matriz_participacion)


La solución final es: (5, 1, 0, 2, 4, 6, 7, 3, 8, 9) en 69690 iteraciones para dimension: 10


In [5]:
## DESARROLLO DEL PROBLEMA II - FUNCION MAIN

dim = len(matriz_participacion[0])
sol_optima = tuple(i for i in range(dim))
cota_superior = f_coste(sol_optima, matriz_participacion)
nodos = []
nodos.append({'s': (), 'ci': f_cota_inferior((), matriz_participacion)})
iteracion = 0
# Inicialización
while nodos:
    iteracion += 1

    # Seleccionar el nodo prometedor con la menor cota inferior
    nodo_prometedor = min(nodos, key=lambda x: x['ci'])['s']

    # Generar los hijos
    hijos = [{'s': x['s'], 'ci': f_cota_inferior(x['s'], matriz_participacion)} for x in crear_hijos(nodo_prometedor, dim)]

    # Revisar la cota superior y actualizar la solución óptima si se encuentra una solución final
    nodo_final = next((x for x in hijos if len(x['s']) == dim), None)
    if nodo_final and nodo_final['ci'] < cota_superior:
        cota_superior = nodo_final['ci']
        sol_optima = nodo_final['s']

    # Poda de los hijos
    hijos = [x for x in hijos if x['ci'] < cota_superior]

    # Añadir los hijos
    nodos.extend(hijos)

    # Eliminar el nodo ramificado
    nodos = [x for x in nodos if x['s'] != nodo_prometedor]

# Resultado final
print("La solución final es:", sol_optima, "en", iteracion, "iteraciones", "para dimension:", dim)


La solución final es: (5, 1, 0, 2, 4, 6, 7, 3, 8, 9) en 69690 iteraciones para dimension: 10


In [ ]:
## IMPORTAMOS LIBRERIAS
import pandas as pd
from google.colab import files
import numpy as np
import random

## CARGA Y VISUALIZACION DEL DATASET

# Se importa el archivo .csv
csv_file = files.upload()

# Paso necesario para poder leer el csv de forma correcta
dataframe = list(csv_file.keys())[0]

# Cargar, procesar y visualizar el dataframe
df = pd.read_csv(dataframe, index_col=0)
# Se elimina la columna 11 ya que es una columna vacia que no aporta información
df = df.drop('Unnamed: 11', axis=1)
print(df)

## DESARROLLO DEL PROBLEMA I - CREACION DE FUNCIONES AUXILIARES

matriz_participacion = df.iloc[1:30, 0:10].astype(int).values
solucion_parcial_elegida = np.array([2, 9, 0, 6, 4, 8, 5, 3, 7, 1])

def f_coste(sol,matriz):
  val = 0
  for i in range (len(sol)):
    val += matriz[sol[i]][i]
  return val

def f_cota_inferior(sol,matriz):
  val = 0
  for i in range (len(sol)):
    val += matriz[i][sol[i]]

  for i in range (len(sol), matriz.shape[1]):
    val += np.min(matriz[:, 1])

  return val

def crear_hijos(nodo, n):
  hijos = []
  for i in range(n):
    if i not in nodo:
      hijos.append({'s': nodo + (i,)})
  return hijos

  ## DESARROLLO DEL PROBLEMA II - FUNCION MAIN

dim = len(matriz_participacion[0])
sol_optima = tuple(i for i in range(dim))
cota_superior = f_coste(sol_optima, matriz_participacion)
nodos = []
nodos.append({'s': (), 'ci': f_cota_inferior((), matriz_participacion)})
iteracion = 0
# Inicialización
while nodos:
    iteracion += 1

    # Seleccionar el nodo prometedor con la menor cota inferior
    nodo_prometedor = min(nodos, key=lambda x: x['ci'])['s']

    # Generar los hijos
    hijos = [{'s': x['s'], 'ci': f_cota_inferior(x['s'], matriz_participacion)} for x in crear_hijos(nodo_prometedor, dim)]

    # Revisar la cota superior y actualizar la solución óptima si se encuentra una solución final
    nodo_final = next((x for x in hijos if len(x['s']) == dim), None)
    if nodo_final and nodo_final['ci'] < cota_superior:
        cota_superior = nodo_final['ci']
        sol_optima = nodo_final['s']

    # Poda de los hijos
    hijos = [x for x in hijos if x['ci'] < cota_superior]

    # Añadir los hijos
    nodos.extend(hijos)

    # Eliminar el nodo ramificado
    nodos = [x for x in nodos if x['s'] != nodo_prometedor]

# Resultado final
print("La solución final es:", sol_optima, "en", iteracion, "iteraciones", "para dimension:", dim)

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Al tener los siguientes datos:


*   10 actores
*   30 tomas

Se concluye que el espacio de soluciones es igual a 30^10. El hecho de que el espacio sea tan grande resalta la necesidad de algoritmos con una buena eficiencia, como puede ser el empleado en este caso, el de ramificación y poda. Esta técnica es muy útil en estos casos porque permite reducir la búsqueda de soluciones. El orden de complejidad de este problema es O(n!) por ser la expresión del número de soluciones una expresión factorial.



#Diseño
- ¿Que técnica utilizo? ¿Por qué?


Como se especifica en el apartado anterior, este problema tiene un espacio de soluciones muy grande, por lo que se necesita un algoritmo que permita reducir la búsqueda de soluciones. Por eso, y por su eficiencia, se utiliza el algoritmo de ramificación y poda.